In [12]:
'''
collect the results from results.pkl or results_gt.pkl
and generate the numbers for table1
'''

import pickle
import os
from glob import glob
import numpy as np

import sys
sys.path.append('../src')

comb_path = '/disk2/data/UE5sim_3DHPE/'

ress = {}
files = os.listdir(comb_path)
for f in files:
    # grab the experimental name and prep its folder.
    fname = f.split('/')[-1].split('.')[0]
    unreal_path = os.path.join(comb_path, fname)

    results_file = os.path.join(unreal_path, 'results_gt.pkl')
#     results_file = os.path.join(unreal_path, 'results.pkl')

    if not os.path.exists(results_file):
        continue
    ress[fname] = pickle.load(open(results_file, 'rb'))
print(len(ress))

31


In [13]:
# calculate score for metrabs vs. lift!
from lift3d_util import *
skip_idc = [7, 9, 10]
from pprint import pprint
good_idc = list(np.arange(17))
for si in skip_idc:
    good_idc.remove(si)
    
leg_idc = set()
for name, idc in limb_chains.items():
    if 'leg' in name:
        leg_idc = leg_idc.union(set(idc))
        
leg_idc = list(leg_idc)

collected_error = defaultdict(lambda: defaultdict(list))

cnt = 0
for fname, compare_dict in ress.items():
    compare_data = compare_dict['skels']
    method_to_err = defaultdict(list)
    cnt += len(compare_data['gt'])
    for key in compare_data.keys():
        if key == 'gt':
            continue
        for frame in range(len(compare_data[key])):
            dists = np.linalg.norm(compare_data[key][frame] - compare_data['gt'][frame], axis=1)[good_idc]
            dists_legs = np.linalg.norm(compare_data[key][frame] - compare_data['gt'][frame], axis=1)[leg_idc]
            method_to_err[key].append([np.mean(dists), np.mean(dists_legs)])
    
    for i, name in enumerate(['total', 'legs']):
        for k, v in method_to_err.items():
            collected_error[name][k].append([np.mean(v[i]), np.std(v[i])])

In [14]:
print('cnt', cnt)

cnt 10571


In [15]:
pixel_errs = defaultdict(list)
for fname, compare_dict in ress.items():
    for k, v in compare_dict['projection_errors'].items():
        pixel_errs[k].append([np.mean(v), np.std(v)])

print('pixel errors')
for k, v in pixel_errs.items():
    print(k, np.mean(v), np.std(v))

pixel errors
metrabs 4.881887683049985 2.88268462109971
metrabs_norot 6.228303241816721 4.067821840353877
lift3d 2.1594741400193707 1.3165017558827485


In [16]:
print('3d error')
for k in method_to_err.keys():
    print(k, np.mean(collected_error['total'][k], 0)*100)

3d error
metrabs [7.81547909 2.08962617]
metrabs_norot [10.26397619  1.57507011]
lift3d [6.1863332  1.50908652]


In [17]:
# compute the error in knee angle!
for fname, compare_dict in ress.items():
    compare_data = compare_dict['skels']
    method_to_err = defaultdict(list)
    for key in compare_data.keys():
        if key == 'gt':
            continue
        for frame in range(len(compare_data[key])):
            pass

In [18]:
# knee_angle_from_lines(compare_data['gt'][0])

left_idc = limb_chains['left_leg'][1:]
right_idc = limb_chains['right_leg'][1:]

err_collect = defaultdict(list)
for fname, compare_dict in ress.items():
    compare_data = compare_dict['skels']
    method_to_err = defaultdict(list)
    for key in compare_data.keys():
        if key == 'gt':
            continue
        for frame in range(len(compare_data[key])):
            
            for side in [left_idc, right_idc]:
                hip, knee, ankle = compare_data['gt'][frame][side]
                knee_angle_gt = angle_between(knee, hip, ankle)
                u = knee - ankle
                u/=np.linalg.norm(u)

                hip, knee, ankle = compare_data[key][frame][side]
                knee_angle_method = angle_between(knee, hip, ankle)
                
                v = knee - ankle
                v/=np.linalg.norm(v)
                shin_off = np.arccos(u.dot(v)) / np.pi * 180
            
                diff = np.abs(knee_angle_gt - knee_angle_method)
                method_to_err[key].append(shin_off)

    for k, v in method_to_err.items():
        err_collect[k].append([np.mean(v), np.std(v)])
#         print(k, np.mean(v), np.std(v))

print('knee error')
for k, v in err_collect.items():
    print(k, np.mean(v, 0))

knee error
metrabs [12.28557895  7.97681484]
metrabs_norot [20.33231988  9.74034112]
lift3d [9.59782521 8.90834991]


In [19]:
for k, v in ress['ath0_run']['skels'].items():
    print(k, np.mean(v), np.std(v))

gt 8.103235552872174 12.803080682360582
metrabs 7.052947601169807 11.38696140771269
metrabs_norot 7.060339719642375 11.377178421809953
lift3d 5.782306542431986 10.669316904551637
